In [1]:
#Import pandas, matplotlib.pyplot, and seaborn
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
from datetime import datetime
import os
import yfinance as yf
from tqdm import tqdm
%matplotlib inline
import scipy.stats
import numpy as np
import statsmodels.api as sm
from statsmodels.graphics.api import abline_plot
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn import linear_model, preprocessing
import nltk 
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import re
from scipy import stats
from statsmodels.tsa.stattools import adfuller
from wordcloud import WordCloud, STOPWORDS
import random


In [2]:
myfold = os.getcwd()

### Load the data

In [3]:
retail_df = pd.read_excel(myfold + '/Online Retail.xlsx')

In [4]:
retail_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    541909 non-null  object        
 1   StockCode    541909 non-null  object        
 2   Description  540455 non-null  object        
 3   Quantity     541909 non-null  int64         
 4   InvoiceDate  541909 non-null  datetime64[ns]
 5   UnitPrice    541909 non-null  float64       
 6   CustomerID   406829 non-null  float64       
 7   Country      541909 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 33.1+ MB


In [5]:
retail_df.head(10)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
5,536365,22752,SET 7 BABUSHKA NESTING BOXES,2,2010-12-01 08:26:00,7.65,17850.0,United Kingdom
6,536365,21730,GLASS STAR FROSTED T-LIGHT HOLDER,6,2010-12-01 08:26:00,4.25,17850.0,United Kingdom
7,536366,22633,HAND WARMER UNION JACK,6,2010-12-01 08:28:00,1.85,17850.0,United Kingdom
8,536366,22632,HAND WARMER RED POLKA DOT,6,2010-12-01 08:28:00,1.85,17850.0,United Kingdom
9,536367,84879,ASSORTED COLOUR BIRD ORNAMENT,32,2010-12-01 08:34:00,1.69,13047.0,United Kingdom


Function to show the analysis of general characteristics of the data

In [6]:
def show_charecteristics(df):
    recs = df.shape[0]
    types = df.dtypes
    counts = df.apply(lambda x: x.count())
    uniques = df.apply(lambda x: x.unique())
    nulls = df.apply(lambda x: x.isnull().sum())
    distincts = df.apply(lambda x: x.unique().shape[0])
    missing_percent =  (df.isnull().sum()/ recs) * 100
    print('Data shape:', df.shape)
    cols = ['types', 'counts', 'distincts', 'nulls','uniques','missing_percent']
    strd = pd.concat([ types,counts, distincts, nulls,uniques,missing_percent], axis = 1, sort=True)
    strd.columns = cols
    print('___________________________\nData types:\n',strd.types.value_counts())
    print('___________________________')

    return strd


### First look at data, its size and metadata

In [7]:
details = show_charecteristics(retail_df).sort_values(by='missing_percent',ascending=False)
details

Data shape: (541909, 8)
___________________________
Data types:
 object            4
float64           2
int64             1
datetime64[ns]    1
Name: types, dtype: int64
___________________________


,types,counts,distincts,nulls,uniques,missing_percent
CustomerID,float64,406829,4373,135080,"[17850.0, 13047.0, 12583.0, 13748.0, 15100.0, ...",24.926694
Description,object,540455,4224,1454,"[WHITE HANGING HEART T-LIGHT HOLDER, WHITE MET...",0.268311
Country,object,541909,38,0,"[United Kingdom, France, Australia, Netherland...",0.000000
InvoiceDate,datetime64[ns],541909,23260,0,"[2010-12-01T08:26:00.000000000, 2010-12-01T08:...",0.000000
InvoiceNo,object,541909,25900,0,"[536365, 536366, 536367, 536368, 536369, 53637...",0.000000
Quantity,int64,541909,722,0,"[6, 8, 2, 32, 3, 4, 24, 12, 48, 18, 20, 36, 80...",0.000000
StockCode,object,541909,4070,0,"[85123A, 71053, 84406B, 84029G, 84029E, 22752,...",0.000000
UnitPrice,float64,541909,1630,0,"[2.55, 3.39, 2.75, 7.65, 4.25, 1.85, 1.69, 2.1...",0.000000


**Let's see the description of each column:**


InvoiceNo: Invoice number. Nominal, a 6-digit integral number uniquely assigned to each transaction. If this code starts with letter 'c', it indicates a cancellation.

StockCode: Product (item) code. Nominal, a 5-digit integral number uniquely assigned to each distinct product.
Description: Product (item) name. Nominal.

Quantity: The quantities of each product (item) per transaction. Numeric.

InvoiceDate: Invoice Date and time. Numeric, the day and time when each transaction was generated.

UnitPrice: Unit price. Numeric, Product price per unit in sterling.

CustomerID: Customer number. Nominal, a 5-digit integral number uniquely assigned to each customer.

Country: Country name. Nominal, the name of the country where each customer resides.


### Clean the dataset & fill up missing values

In [8]:
retail_df.describe()

,Quantity,UnitPrice,CustomerID
count,541909.000000,541909.000000,406829.000000
mean,9.552250,4.611114,15287.690570
std,218.081158,96.759853,1713.600303
min,-80995.000000,-11062.060000,12346.000000
25%,1.000000,1.250000,13953.000000
50%,3.000000,2.080000,15152.000000
75%,10.000000,4.130000,16791.000000
max,80995.000000,38970.000000,18287.000000


Quantity filed has negative values. Similarly unit price filed has negative vales. These trasanctions might include returns as well. As our goal is to perform customer segmentation and market basket analysis, we can remove these records. Also, lets invetigate further and see whether there are records where both are negative or if one of them is negative and the other is zero.

In [9]:
print('Is there any negative quantity and prices at same register:','No' if 
      retail_df[(retail_df.Quantity < 0) & (retail_df.UnitPrice < 0)].shape[0]==0  else'Yes')
print('\nAre there any registers where quantity is negative and price is zero or vice-versa:',
      retail_df[(retail_df.Quantity <= 0) & (retail_df.UnitPrice <= 0)].shape[0])
print('\nIs there a  customer ID for the above records: ' ,retail_df.loc[(retail_df.Quantity<=0) & (retail_df.UnitPrice<=0), 
                ['CustomerID']].CustomerID.unique())

Is there any negative quantity and prices at same register: No

Are there any registers where quantity is negative and price is zero or vice-versa: 1336

Is there a  customer ID for the above records:  [nan]


In [10]:
print('\nSample of records with negative and positive quantity: ') 
display(retail_df[(retail_df.CustomerID ==12472) & (retail_df.StockCode == 22244)])

print('\nAll register with negative quantity has Invoice start with:',
      retail_df.loc[(retail_df['Quantity'] <= 0) & (retail_df['CustomerID'].notnull()),'InvoiceNo'].apply(lambda x: x[0]).unique())
                                      


Sample of records with negative and positive quantity: 


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
1973,C536548,22244,3 HOOK HANGER MAGIC GARDEN,-4,2010-12-01 14:33:00,1.95,12472.0,Germany
9438,537201,22244,3 HOOK HANGER MAGIC GARDEN,12,2010-12-05 14:19:00,1.95,12472.0,Germany
121980,546843,22244,3 HOOK HANGER MAGIC GARDEN,12,2011-03-17 12:40:00,1.95,12472.0,Germany



All register with negative quantity has Invoice start with: ['C']


In [11]:
print('Records that have UnitPrice negative:', retail_df.loc[retail_df.UnitPrice < 0].shape[0])
display(retail_df.loc[retail_df['UnitPrice'] < 0])

Records that have UnitPrice negative: 2


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
299983,A563186,B,Adjust bad debt,1,2011-08-12 14:51:00,-11062.06,NaN,United Kingdom
299984,A563187,B,Adjust bad debt,1,2011-08-12 14:52:00,-11062.06,NaN,United Kingdom


In [12]:
print("Sales records with Customer ID and zero in Unit Price:",retail_df[(retail_df.UnitPrice==0)  & (retail_df.CustomerID.notnull())].shape[0])
retail_df[(retail_df.UnitPrice==0)  & (retail_df.CustomerID.notnull())]

Sales records with Customer ID and zero in Unit Price: 40


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
9302,537197,22841,ROUND CAKE TIN VINTAGE GREEN,1,2010-12-05 14:02:00,0.0,12647.0,Germany
33576,539263,22580,ADVENT CALENDAR GINGHAM SACK,4,2010-12-16 14:36:00,0.0,16560.0,United Kingdom
40089,539722,22423,REGENCY CAKESTAND 3 TIER,10,2010-12-21 13:45:00,0.0,14911.0,EIRE
47068,540372,22090,PAPER BUNTING RETROSPOT,24,2011-01-06 16:41:00,0.0,13081.0,United Kingdom
47070,540372,22553,PLASTERS IN TIN SKULLS,24,2011-01-06 16:41:00,0.0,13081.0,United Kingdom
56674,541109,22168,ORGANISER WOOD ANTIQUE WHITE,1,2011-01-13 15:10:00,0.0,15107.0,United Kingdom
86789,543599,84535B,FAIRY CAKES NOTEBOOK A6 SIZE,16,2011-02-10 13:08:00,0.0,17560.0,United Kingdom
130188,547417,22062,CERAMIC BOWL WITH LOVE HEART DESIGN,36,2011-03-23 10:25:00,0.0,13239.0,United Kingdom
139453,548318,22055,MINI CAKE STAND HANGING STRAWBERY,5,2011-03-30 12:45:00,0.0,13113.0,United Kingdom
145208,548871,22162,HEART GARLAND RUSTIC PADDED,2,2011-04-04 14:42:00,0.0,14410.0,United Kingdom


In [13]:
retail_df[retail_df.CustomerID.isnull()].shape[0]

135080

* There are no records where both quantity and price are negative.

* There are 1336 records where one of them is and the other is 0. It is to be noted here, that for all these records, we do not have the customer ID. 

* We can conclude to delete all records in that quantity or the price is negative. 

* We also see that there are 135,080 records without customer identification that we may disregard.

In [14]:
#Remove records without CustomerID

retail_df = retail_df[~(retail_df.CustomerID.isnull())]

# Remove records where quantity is negative
retail_df = retail_df[~(retail_df.Quantity<0)]

# Remove records where unit price is negative
retail_df = retail_df[retail_df.UnitPrice>0]

In [15]:
details = show_charecteristics(retail_df).sort_values(by='distincts', ascending=False)
details

Data shape: (397884, 8)
___________________________
Data types:
 object            4
float64           2
int64             1
datetime64[ns]    1
Name: types, dtype: int64
___________________________


,types,counts,distincts,nulls,uniques,missing_percent
InvoiceNo,object,397884,18532,0,"[536365, 536366, 536367, 536368, 536369, 53637...",0.0
InvoiceDate,datetime64[ns],397884,17282,0,"[2010-12-01T08:26:00.000000000, 2010-12-01T08:...",0.0
CustomerID,float64,397884,4338,0,"[17850.0, 13047.0, 12583.0, 13748.0, 15100.0, ...",0.0
Description,object,397884,3877,0,"[WHITE HANGING HEART T-LIGHT HOLDER, WHITE MET...",0.0
StockCode,object,397884,3665,0,"[85123A, 71053, 84406B, 84029G, 84029E, 22752,...",0.0
UnitPrice,float64,397884,440,0,"[2.55, 3.39, 2.75, 7.65, 4.25, 1.85, 1.69, 2.1...",0.0
Quantity,int64,397884,301,0,"[6, 8, 2, 32, 3, 4, 24, 12, 48, 18, 20, 36, 80...",0.0
Country,object,397884,37,0,"[United Kingdom, France, Australia, Netherland...",0.0


We can see that the stockcode and description does not matchup which can be due to data entry issues.

In [16]:
desc_df = retail_df[['StockCode','Description']].value_counts().reset_index()
display(desc_df.StockCode.value_counts()[desc_df.StockCode.value_counts()>1].reset_index().head(4))

,index,StockCode
0,23236,4
1,23196,4
2,23413,3
3,23240,3


In [17]:
retail_df[retail_df['StockCode'] == desc_df.StockCode.value_counts()[desc_df.StockCode.value_counts()>1]
      .reset_index()['index'][4]]['Description'].unique()

array(['JUMBO BAG DOILEY PATTERNS', 'JUMBO BAG VINTAGE DOILEY ',
       'JUMBO BAG VINTAGE DOILY '], dtype=object)

We can see some of the items have multiple descriptions. A simple spelling mistake or an additional space can end up in reducing data quality.Let's fix it by using the most frequently used description.

In [18]:
unique_desc  = retail_df.groupby('StockCode')['Description'].apply(lambda x: x.mode().iloc[0]).reset_index()
retail_df['Description'] = retail_df['StockCode'].map(unique_desc.set_index('StockCode')['Description'])

In [19]:
details = show_charecteristics(retail_df).sort_values(by='distincts', ascending=False)
display(details)

Data shape: (397884, 8)
___________________________
Data types:
 object            4
float64           2
int64             1
datetime64[ns]    1
Name: types, dtype: int64
___________________________


,types,counts,distincts,nulls,uniques,missing_percent
InvoiceNo,object,397884,18532,0,"[536365, 536366, 536367, 536368, 536369, 53637...",0.0
InvoiceDate,datetime64[ns],397884,17282,0,"[2010-12-01T08:26:00.000000000, 2010-12-01T08:...",0.0
CustomerID,float64,397884,4338,0,"[17850.0, 13047.0, 12583.0, 13748.0, 15100.0, ...",0.0
StockCode,object,397884,3665,0,"[85123A, 71053, 84406B, 84029G, 84029E, 22752,...",0.0
Description,object,397884,3647,0,"[WHITE HANGING HEART T-LIGHT HOLDER, WHITE MET...",0.0
UnitPrice,float64,397884,440,0,"[2.55, 3.39, 2.75, 7.65, 4.25, 1.85, 1.69, 2.1...",0.0
Quantity,int64,397884,301,0,"[6, 8, 2, 32, 3, 4, 24, 12, 48, 18, 20, 36, 80...",0.0
Country,object,397884,37,0,"[United Kingdom, France, Australia, Netherland...",0.0


In [20]:
retail_df.InvoiceDate = pd.to_datetime(retail_df.InvoiceDate)
retail_df.CustomerID = retail_df.CustomerID.astype('int64')

# Add a colmn amount
retail_df['Amount'] = retail_df['Quantity'] * retail_df['UnitPrice']

In [21]:
details = show_charecteristics(retail_df).sort_values(by='distincts', ascending=False)
display(details)

Data shape: (397884, 9)
___________________________
Data types:
 object            4
int64             2
float64           2
datetime64[ns]    1
Name: types, dtype: int64
___________________________


,types,counts,distincts,nulls,uniques,missing_percent
InvoiceNo,object,397884,18532,0,"[536365, 536366, 536367, 536368, 536369, 53637...",0.0
InvoiceDate,datetime64[ns],397884,17282,0,"[2010-12-01T08:26:00.000000000, 2010-12-01T08:...",0.0
CustomerID,int64,397884,4338,0,"[17850, 13047, 12583, 13748, 15100, 15291, 146...",0.0
StockCode,object,397884,3665,0,"[85123A, 71053, 84406B, 84029G, 84029E, 22752,...",0.0
Description,object,397884,3647,0,"[WHITE HANGING HEART T-LIGHT HOLDER, WHITE MET...",0.0
Amount,float64,397884,2939,0,"[15.299999999999999, 20.34, 22.0, 15.3, 25.5, ...",0.0
UnitPrice,float64,397884,440,0,"[2.55, 3.39, 2.75, 7.65, 4.25, 1.85, 1.69, 2.1...",0.0
Quantity,int64,397884,301,0,"[6, 8, 2, 32, 3, 4, 24, 12, 48, 18, 20, 36, 80...",0.0
Country,object,397884,37,0,"[United Kingdom, France, Australia, Netherland...",0.0


### Save the cleaned data to csv

In [23]:
retail_df.to_csv('online_retail_cleaned.csv')

### Summary

Now the dataset is clean.
It has 397884 rows. There are no null values. We will now move to exploratory data analysis.